### 📄 INSTALLATION PACKAGES

#%pip install notebook ipykernel

In [1]:
# === LLM & LangChain Integration ===
#%pip install langchain
#%pip install langchain-openai
#%pip install -U langchain-ollama

# Initialize LLMs

In [ ]:
from langchain_ollama import ChatOllama

deepseekcloud_llm = ChatOllama(
    model="deepseek-v3.1:671b-cloud",        
    base_url="http://localhost:11434",
    temperature=0.5
)


In [3]:
#%pip install langchain-anthropic

In [4]:
from langchain_anthropic import ChatAnthropic  # or suitable class
import os

os.environ["ANTHROPIC_API_KEY"] = "your_api_key_here"

claude_llm = ChatAnthropic(
    model="claude-sonnet-4",  # example model
    temperature=0.5
)

In [5]:
#%pip show langchain
#%pip install --upgrade langchain
#%pip install -qU langchain-community unstructured

### 📝 Generate BDD Test Cases from PDF

In [6]:
#%pip install langchain-document-loaders

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import UnstructuredPDFLoader

def generate_bdd_test_cases_from_pdf(user_story: str) -> str:
    """
    📄 Reads requirements from a specific PDF file and generates comprehensive 
    BDD test scenarios in Gherkin format.
    Includes valid, invalid, edge case, and alternative flows.
    """

    pdf_file = "./Docs/LoginDocumentation.pdf"
    loader = UnstructuredPDFLoader(
    file_path=pdf_file,
    languages=["eng"]  # explicitly specify English for OCR
    )
    docs = loader.load()  # returns list of Document objects

    # Combine text from all pages (limit to first 2000 chars for safety)
    requirements_text = "\n\n".join([doc.page_content.strip() for doc in docs])[:2000]

    prompt_template = PromptTemplate.from_template(
        """
        You are a QA Automation Engineer. 
        Your task is to convert the following user story into ALL possible test cases 
        in Gherkin BDD style format.
        Include valid, invalid, edge case, and alternative flow scenarios.

        {requirements_text}

        Format your response as:
        Feature: [Feature name]

        Scenario: [Scenario name]
            Given [precondition]
            When [action]
            Then [expected result]
        """
    )

    prompt = prompt_template.format(requirements_text=requirements_text)
    response = llama_llm.invoke(prompt)

    # Extract content from the response
    if hasattr(response, 'content'):
        return response.content
    else:
        return str(response)

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import UnstructuredPDFLoader

def generate_single_bdd_test_case_from_pdf(user_story: str) -> str:
    """
    📄 Reads requirements from a specific PDF file and generates a single 
    BDD test scenario in Gherkin format.
    Includes a combination of valid, invalid, edge case, or alternative flows.
    """

    pdf_file = "./Docs/LoginDocumentation.pdf"
    loader = UnstructuredPDFLoader(
    file_path=pdf_file,
    languages=["eng"]  # explicitly specify English for OCR
    )
    docs = loader.load()

    # Combine text from all pages (limit to first 2000 chars for safety)
    requirements_text = "\n\n".join([doc.page_content.strip() for doc in docs])[:2000]

    prompt_template = PromptTemplate.from_template(
        """
        You are a QA Automation Engineer. 
        Your task is to convert the following user story into ONLY ONE test case in Gherkin BDD style format.

        {requirements_text}

        Format your response as:
        Feature: [Feature name]

        Scenario: [Scenario name]
            Given [precondition]
            When [action]
            Then [expected result]
        """
    )

    prompt = prompt_template.format(requirements_text=requirements_text)
    response = deepseekcloud_llm.invoke(prompt)

    # Extract content from the response
    if hasattr(response, 'content'):
        return response.content
    else:
        return str(response)


In [9]:
# Make sure your tool is imported
# from your_module import generate_bdd_test_cases_from_pdf
# and your LLM instance: qwen_llm

# Input for the tool
user_story_input = "Generate BDD test cases from PDF"

# Directly call the tool
try:
    result = generate_single_bdd_test_case_from_pdf(user_story_input)
    print("📄 Generated BDD Test Cases from PDF:\n")
    print(result)
except Exception as e:
    print(f"Error occurred: {e}")

📄 Generated BDD Test Cases from PDF:

Feature: OrangeHRM Login Authentication

Scenario: Successful login with valid demo credentials
    Given I am on the OrangeHRM login page
    When I enter "Admin" in the username field and "admin123" in the password field
    And I click the login button
    Then I should be redirected to the OrangeHRM dashboard
